# Complete candidate search (nrw_2022, 5,000 sequences)
This notebook contains the evaluation of the complete candidate search concerning the aggregate due_2022 with 5,000 sequences. Breadth search and depth search results were taken into account.
The distance matrices were evaluated in terms of infection recall. Furthermore, the mst structure was evaluated based on community ARI, lineage purity and mean edge weight. MSTs were generated to compare the results with the MST of the optimized algorithm regarding the distribution of outbreak-related attributes.

In [1]:
!pip install ../../gentrain/.

Processing /Users/benkraling/code/thesis/gentrain
  Preparing metadata (setup.py) ... done
  Created wheel for gentrain: filename=gentrain-0.1.2-py3-none-any.whl size=26677 sha256=16241c3b3bc2dafcfc1099a4f1f8d5a9973f4d8cbf9458f521d73895b7d8adb7
  Stored in directory: /private/var/folders/2h/923cq6912sqb0snfvqqfdnmm0000gn/T/pip-ephem-wheel-cache-f_znmrgd/wheels/cf/e4/57/91c03db2e8c043adeefe35dd0969d3049f61ae0218be0acc9f
Successfully built gentrain
  Attempting uninstall: gentrain
    Found existing installation: gentrain 0.1.2
    Uninstalling gentrain-0.1.2:
      Successfully uninstalled gentrain-0.1.2


In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle
from gentrain.evaluation import get_computation_rate_plot, candidate_evaluation_and_matrices, get_candidate_evaluation_and_export_mst
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from gentrain.encoding import get_nucleotide_sensitive_encodings, get_mutation_sensitive_encodings, generate_one_hot_encoding
from gentrain.nextclade import get_mutations_from_dataframe
from gentrain.candidate_sourcing import bitwise_xor_candidates
from gentrain.graph import build_mst, export_graph_gexf, mean_edge_weight, get_outbreak_community_labels, build_graph
from scipy.spatial.distance import pdist
import community as community_louvain
import umap
import faiss
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances_argmin
import os
import shutil

In [3]:
aggregate = "nrw_2022"
size = 5000

In [4]:
graph_path = f"graphs/{aggregate}/{size}"
if os.path.exists(f"graphs/{aggregate}/{size}") and os.path.isdir(f"graphs/{aggregate}/{size}"):
    shutil.rmtree(f"graphs/{aggregate}/{size}")
if not os.path.exists(f"graphs/{aggregate}"):
    os.mkdir(f"graphs/{aggregate}")
os.mkdir(f"graphs/{aggregate}/{size}")

In [5]:
sequences_df = pd.read_csv(f"../00_data_understanding_and_preparation/aggregates/{aggregate}/{size}/sequences_and_metadata.csv", delimiter=";").set_index("igs_id").sort_index()
sequences_count = len(sequences_df)

In [6]:
mutations_df = get_mutations_from_dataframe(sequences_df)

In [7]:
distance_matrix_df = pd.read_csv(f"../01_algorithm_optimization/distance_matrices/{aggregate}/{size}/distance_matrix.csv", delimiter=";",
                                     index_col="Unnamed: 0").sort_index()
distance_matrix_df = distance_matrix_df[~distance_matrix_df.index.duplicated(keep="first")]
distance_matrix_df = distance_matrix_df.loc[sequences_df.index, sequences_df.index]
distance_matrix = distance_matrix_df.to_numpy()

In [8]:
gentrain_graph = build_graph(distance_matrix)
gentrain_mst = build_mst(gentrain_graph)
gentrain_community_labels = get_outbreak_community_labels(gentrain_mst)
datetime_sampling_dates = pd.to_datetime(sequences_df["date_of_sampling"])
numeric_dates = (datetime_sampling_dates - datetime_sampling_dates.min()).dt.days
export_graph_gexf(gentrain_mst, gentrain_community_labels, sequences_df, f"{graph_path}/brute_force")

mst generation time: 94.67s


In [9]:
mask = np.triu(np.ones(distance_matrix_df.shape), k=1).astype(bool)
filtered = distance_matrix_df.where(mask)
infections_count = (filtered < 2).sum().sum()
distances_count = filtered.count().sum()

## Encoding

In [10]:
encodings_N_frequency_filter = get_mutation_sensitive_encodings(sequences_df, exclude_indels=False, use_frequency_filtering=False, filter_N=True)

execution time: 73.74s


In [11]:
encodings_N_and_SNV_frequency_filter = get_mutation_sensitive_encodings(sequences_df, exclude_indels=False, use_frequency_filtering=True, filter_N=True)

execution time: 71.24s


## Accurate candidate search

### Depth search with N frequency filter

In [12]:
depth_search_N_frequency_filter = []
for computation_rate in [0.05, 0.1, 0.15, 0.2]:
    limit = int(computation_rate*distances_count)
    candidates, runtime = bitwise_xor_candidates(encodings_N_frequency_filter, limit, "depth")
    depth_search_N_frequency_filter.append(get_candidate_evaluation_and_export_mst("depth_N", candidates, graph_path, distance_matrix, gentrain_community_labels, gentrain_mst, list(sequences_df["clade"]), sequences_df, runtime))

execution time xor distance calculation: 86.37s
execution time depth search: 8.95s
execution time 624875: 98.07s
mst generation time: 4.95s
execution time xor distance calculation: 86.05s
execution time depth search: 10.32s
execution time 1249750: 98.15s
mst generation time: 6.7s
execution time xor distance calculation: 84.66s
execution time depth search: 11.18s
execution time 1874625: 97.79s
mst generation time: 14.25s
execution time xor distance calculation: 84.21s
execution time depth search: 10.18s
execution time 2499500: 96.45s
mst generation time: 17.46s


In [13]:
pd.DataFrame(depth_search_N_frequency_filter)

,computation_rate,infection_detection_rate,infection_recall,infection_precision,infection_f1,runtime,mean_edge_weight,mean_edge_weight_diff,max_edge_weight,subgraph_count,adjusted_rand_index,lineage_purity,lineage_purity_diff
0,0.05,0.912987,1.0,1.0,0.912987,98.07,3.233542,-0.194444,15.0,358,0.372452,0.983308,0.028308
1,0.10,0.969776,1.0,1.0,0.969776,98.15,3.331951,-0.096034,19.0,152,0.626862,0.968332,0.013332
2,0.15,0.986541,1.0,1.0,0.986541,97.79,3.314154,-0.113831,19.0,111,0.776623,0.959975,0.004975
3,0.20,0.990909,1.0,1.0,0.990909,96.45,3.337897,-0.090089,22.0,84,0.817148,0.954268,-0.000732


### Breadth search with N frequency filter

In [14]:
breadth_search_N_frequency_filter = []
for computation_rate in [0.05, 0.1, 0.15, 0.2]:
    limit = int(computation_rate*distances_count)
    candidates, runtime = bitwise_xor_candidates(encodings_N_frequency_filter, limit, "breadth")
    breadth_search_N_frequency_filter.append(get_candidate_evaluation_and_export_mst("breadth_N", candidates, graph_path, distance_matrix, gentrain_community_labels, gentrain_mst, list(sequences_df["clade"]), sequences_df, runtime))

matrix generation time: 64.75s
execution time distance collection: 14.53s
execution time breadth search: 12.5s
execution time 624875: 93.78s
mst generation time: 4.34s
matrix generation time: 61.69s
execution time distance collection: 14.47s
execution time breadth search: 12.41s
execution time 1249750: 90.83s
mst generation time: 6.75s
matrix generation time: 63.64s
execution time distance collection: 13.85s
execution time breadth search: 12.34s
execution time 1874625: 91.7s
mst generation time: 10.52s
matrix generation time: 63.99s
execution time distance collection: 13.41s
execution time breadth search: 12.1s
execution time 2499500: 91.38s
mst generation time: 12.71s


In [15]:
pd.DataFrame(breadth_search_N_frequency_filter)

,computation_rate,infection_detection_rate,infection_recall,infection_precision,infection_f1,runtime,mean_edge_weight,mean_edge_weight_diff,max_edge_weight,subgraph_count,adjusted_rand_index,lineage_purity,lineage_purity_diff
0,0.049380,0.775207,1.0,1.0,0.775207,93.78,3.634947,0.206961,74.0,1,0.383423,0.9678,0.0128
1,0.097510,0.880756,1.0,1.0,0.880756,90.83,3.563973,0.135987,74.0,1,0.488546,0.9646,0.0096
2,0.144389,0.928571,1.0,1.0,0.928571,91.70,3.521184,0.093199,74.0,1,0.585905,0.9644,0.0094
3,0.190018,0.951948,1.0,1.0,0.951948,91.38,3.498280,0.070294,74.0,1,0.662671,0.9600,0.0050


### Depth search with N and SNV frequency filter

In [16]:
depth_search_N_and_SNV_frequency_filter = []
for computation_rate in [0.05, 0.1, 0.15, 0.2]:
    limit = int(computation_rate*distances_count)
    candidates, runtime = bitwise_xor_candidates(encodings_N_and_SNV_frequency_filter, limit, "depth")
    depth_search_N_and_SNV_frequency_filter.append(get_candidate_evaluation_and_export_mst("depth_N_and_SNV", candidates, graph_path, distance_matrix, gentrain_community_labels, gentrain_mst, list(sequences_df["clade"]), sequences_df, runtime))

execution time xor distance calculation: 52.0s
execution time depth search: 8.7s
execution time 624875: 62.32s
mst generation time: 3.47s
execution time xor distance calculation: 51.76s
execution time depth search: 8.79s
execution time 1249750: 62.09s
mst generation time: 6.36s
execution time xor distance calculation: 50.66s
execution time depth search: 8.78s
execution time 1874625: 61.01s
mst generation time: 9.52s
execution time xor distance calculation: 50.58s
execution time depth search: 8.84s
execution time 2499500: 60.96s
mst generation time: 14.1s


In [17]:
pd.DataFrame(depth_search_N_and_SNV_frequency_filter)

,computation_rate,infection_detection_rate,infection_recall,infection_precision,infection_f1,runtime,mean_edge_weight,mean_edge_weight_diff,max_edge_weight,subgraph_count,adjusted_rand_index,lineage_purity,lineage_purity_diff
0,0.05,0.942385,1.0,1.0,0.942385,62.32,3.213547,-0.214438,22.0,357,0.422696,0.972921,0.017921
1,0.10,0.978985,1.0,1.0,0.978985,62.09,3.304535,-0.123450,20.0,171,0.601214,0.969247,0.014247
2,0.15,0.992798,1.0,1.0,0.992798,61.01,3.317772,-0.110213,22.0,116,0.770286,0.960163,0.005163
3,0.20,0.994923,1.0,1.0,0.994923,60.96,3.322666,-0.105319,21.0,94,0.853284,0.959715,0.004715


### Breadth search with N and SNV frequency filter

In [18]:
breadth_search_N_and_SNV_frequency_filter = []
for computation_rate in [0.05, 0.1, 0.15, 0.2]:
    limit = int(computation_rate*distances_count)
    candidates, runtime = bitwise_xor_candidates(encodings_N_and_SNV_frequency_filter, limit, "breadth")
    breadth_search_N_and_SNV_frequency_filter.append(get_candidate_evaluation_and_export_mst("breadth_N_and_SNV", candidates, graph_path, distance_matrix, gentrain_community_labels, gentrain_mst, list(sequences_df["clade"]), sequences_df, runtime))

matrix generation time: 33.98s
execution time distance collection: 13.08s
execution time breadth search: 11.65s
execution time 624875: 60.52s
mst generation time: 3.1s
matrix generation time: 34.36s
execution time distance collection: 13.97s
execution time breadth search: 12.39s
execution time 1249750: 62.5s
mst generation time: 7.03s
matrix generation time: 34.26s
execution time distance collection: 13.26s
execution time breadth search: 12.05s
execution time 1874625: 61.47s
mst generation time: 9.55s
matrix generation time: 34.57s
execution time distance collection: 14.17s
execution time breadth search: 12.31s
execution time 2499500: 62.84s
mst generation time: 12.46s


In [19]:
pd.DataFrame(breadth_search_N_and_SNV_frequency_filter)

,computation_rate,infection_detection_rate,infection_recall,infection_precision,infection_f1,runtime,mean_edge_weight,mean_edge_weight_diff,max_edge_weight,subgraph_count,adjusted_rand_index,lineage_purity,lineage_purity_diff
0,0.049380,0.808501,1.0,1.0,0.808501,60.52,3.612482,0.184497,74.0,1,0.389345,0.9634,0.0084
1,0.097510,0.914758,1.0,1.0,0.914758,62.50,3.556531,0.128546,74.0,1,0.492223,0.9622,0.0072
2,0.144389,0.956198,1.0,1.0,0.956198,61.47,3.518604,0.090618,74.0,1,0.591070,0.9590,0.0040
3,0.190018,0.971547,1.0,1.0,0.971547,62.84,3.503081,0.075095,74.0,1,0.653711,0.9586,0.0036


In [20]:
depth_search_evaluation = {
    "N frequency filter": {"values": {evaluation["computation_rate"]: evaluation for evaluation in depth_search_N_frequency_filter}, "stroke": "dash", "color": "black"},
    "N & SNV frequency filter": {"values": {evaluation["computation_rate"]: evaluation for evaluation in depth_search_N_and_SNV_frequency_filter}, "stroke": "dot", "color": "black"}
}

In [21]:
breadth_search_evaluation = {
    "N frequency filter": {"values": {evaluation["computation_rate"]: evaluation for evaluation in breadth_search_N_frequency_filter}, "stroke": "dash", "color": "black"},
    "N & SNV frequency filter": {"values": {evaluation["computation_rate"]: evaluation for evaluation in breadth_search_N_and_SNV_frequency_filter}, "stroke": "dot", "color": "black"}
}

### Infection recall for different filters and computation rates using depth search

In [22]:
sub_fig = get_computation_rate_plot("infection_detection_rate", depth_search_evaluation, "Infection recall", dict(
            x=0.85,
            y=0,
            itemwidth=60,
            xanchor="left",
            yanchor="bottom",
            font=dict(size=30),
        ))

sub_fig.show()

### Community ARI for different filters and computation rates using depth search

In [23]:
sub_fig = get_computation_rate_plot("adjusted_rand_index", depth_search_evaluation, "Community ARI", dict(
            x=0.85,
            y=0,
            itemwidth=60,
            xanchor="left",
            yanchor="bottom",
            font=dict(size=30),
        ))

sub_fig.show()

### Infection recall for different filters and computation rates using breadth search

In [24]:
sub_fig = get_computation_rate_plot("infection_detection_rate", breadth_search_evaluation, "Infection recall", legend=dict(
            x=0.65,
            y=0.05,
            xanchor="left",
            yanchor="bottom",
            font=dict(size=35),
        ))
sub_fig.show()

### Community ARI for different filters and computation rates using breadth search

In [25]:
sub_fig = get_computation_rate_plot("adjusted_rand_index", breadth_search_evaluation, "Community ARI", legend=dict(
            x=0.55,
            y=0.02,
            xanchor="left",
            yanchor="bottom",
            font=dict(size=35),
        ))
sub_fig.show()

In [26]:
depth_vs_breadth_search_evaluation = {
    "Depth search": {"values": {evaluation["computation_rate"]: evaluation for evaluation in depth_search_N_and_SNV_frequency_filter}, "stroke": "solid", "color": "blue"},
    "Breadth search": {"values": {evaluation["computation_rate"]: evaluation for evaluation in breadth_search_N_and_SNV_frequency_filter}, "stroke": "solid", "color": "green"},  
}

### Infection recall for different computation rates using depth search and breadth search (N and SNV frequency filter)

In [27]:
sub_fig = get_computation_rate_plot("infection_detection_rate", depth_vs_breadth_search_evaluation, "Infection recall", legend=dict(
            x=0.7,
            y=0.1,
            xanchor="left",
            yanchor="bottom",
            font=dict(size=35),
        ))
sub_fig.show()

### Community ARI for different computation rates using depth search and breadth search (N and SNV frequency filter)

In [28]:
sub_fig = get_computation_rate_plot("adjusted_rand_index", depth_vs_breadth_search_evaluation, "Community ARI", legend=dict(
            x=0.7,
            y=0.1,
            xanchor="left",
            yanchor="bottom",
            font=dict(size=35),
        ))
sub_fig.show()